In [1]:
import csv
import importlib
import itertools
import operator
import os
import random
import time
from math import e, exp, log, pi, sqrt
import os
import psutil

import keras.layers as layers
import copy
import keras.models
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import np_utils
from numba import guvectorize, jit, njit, prange, vectorize
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

import tensorflow as tf
from os.path import *
import glob
import cnn_builder as cbuild
import cnn_runner as crun
import config
import dr_methods as drm
import feature_interpretation as cnna
import niftiutils.helper_fxns as hf
import niftiutils.private as prv
import niftiutils.transforms as tr
import niftiutils.visualization as vis
import voi_methods as vm
import feature_influence as finf
import niftiutils.deep_learning.common as common

%matplotlib inline
np.set_printoptions(3)

C:\Users\Clinton\AppData\Local\conda\conda\envs\old-keras\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\Clinton\AppData\Local\conda\conda\envs\old-keras\lib\site-packages\dicom\__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


In [3]:
importlib.reload(config)
importlib.reload(hf)
importlib.reload(drm)
importlib.reload(cbuild)
importlib.reload(cnna) 
importlib.reload(crun)
C = config.Config()
T = config.Hyperparams(C.dataset)

### KDE for feature identification

In [4]:
orig_data_dict, num_samples = cbuild._collect_unaug_data()
num_annotations = 10

features_by_cls, feat_count = cnna.collect_features()
feat_count.pop("homogeneous texture")
all_features = sorted(list(feat_count.keys()))
cls_features = {f: [c for c in C.cls_names if f in features_by_cls[c]] for f in all_features}

Z_features = cnna.get_annotated_files(features_by_cls)
Z_features.pop("homogeneous texture")

num_features = len(all_features) # number of features

all_imgs = [orig_data_dict[cls][0] for cls in C.cls_names]
all_imgs = np.array(hf.flatten(all_imgs))

all_lesionids = [orig_data_dict[cls][1] for cls in C.cls_names]
all_lesionids = np.array(hf.flatten(all_lesionids))
test_indices = np.where(np.isin(all_lesionids, C.Z_reader))[0]

x_test = all_imgs[test_indices]
z_test = all_lesionids[test_indices]

full_dfs = []

In [ ]:
R = crun.CNNRunner(C,T)
R.run_fixed_hyperparams(max_runs=2, Z_test=C.Z_reader, model_name='fixZ_std_', verbose=True)

In [5]:
model_prefix = 'fixZ_std_'
model_ix = 1

fullM = keras.models.load_model(join(C.model_dir, model_prefix+"%d.hdf5" % model_ix))
M = keras.models.load_model(join(C.model_dir, model_prefix+"%d.hdf5" % model_ix))
M = common.pop_n_layers(M, 3)

all_dense = cnna.get_overall_activations(M, orig_data_dict)
feature_dense = cnna.get_feature_activations(M, Z_features, all_features)

In [6]:
fullM = keras.models.load_model(join(C.model_dir, model_prefix+"%d.hdf5" % model_ix)) #models_305
model_fc = keras.models.load_model(join(C.model_dir, model_prefix+"%d.hdf5" % model_ix)) #models_305
#model_fc = cbuild.build_cnn_hyperparams(T)
#for l in range(len(model_fc.layers)):
#    model_fc.layers[l].set_weights(M.layers[l].get_weights())
model_fc = common.pop_n_layers(model_fc, 2)

all_dense = cnna.get_overall_activations(model_fc, orig_data_dict)
feature_dense = cnna.get_feature_activations(model_fc, Z_features, all_features)

In [22]:
import math

In [25]:
math.log(math.log(feature_dense['arterial phase enhancement'].shape[0]))

2.0282669849192843

In [44]:
importlib.reload(cnna)
df = cnna.predict_test_features(fullM, model_fc, all_dense, feature_dense, x_test, z_test)

............................................................

In [45]:
df

,true_cls,pred_cls,feature_1,strength_1,feature_2,strength_2,feature_3,strength_3,feature_4,strength_4,...,feature_10,strength_10,feature_11,strength_11,feature_12,strength_12,feature_13,strength_13,feature_14,strength_14
E100894274_0,hcc,hcc,isointense on venous/delayed phase,5.675523e-08,washout,5.673258e-08,arterial phase enhancement,5.673163e-08,enhancing rim/capsule,5.664721e-08,...,nodular or discontinuous enhancement,5.610847e-08,hyperenhancing mass on delayed phase,1.920278e-08,infiltrative growth,8.242306e-09,nodular growth,8.239368e-09,central scar,4.523444e-10
13031955_0,hcc,cholangio,heterogeneous lesion,5.660396e-08,arterial phase enhancement,5.652347e-08,isointense on venous/delayed phase,5.649449e-08,washout,5.649345e-08,...,hypoenhancing core,5.609179e-08,hyperenhancing mass on delayed phase,1.925225e-08,infiltrative growth,8.312716e-09,nodular growth,8.300067e-09,central scar,4.532075e-10
13092966_0,hcc,hcc,washout,5.669515e-08,isointense on venous/delayed phase,5.666462e-08,arterial phase enhancement,5.664975e-08,enhancing rim/capsule,5.659509e-08,...,nodular or discontinuous enhancement,5.610558e-08,hyperenhancing mass on delayed phase,1.920936e-08,nodular growth,8.223439e-09,infiltrative growth,8.219444e-09,central scar,4.508192e-10
E101069048_1,hcc,hcc,washout,5.737838e-08,arterial phase enhancement,5.714853e-08,enhancing rim/capsule,5.700949e-08,isointense on venous/delayed phase,5.688791e-08,...,nodular or discontinuous enhancement,5.636605e-08,hyperenhancing mass on delayed phase,1.930619e-08,infiltrative growth,8.268130e-09,nodular growth,8.265810e-09,central scar,4.547615e-10
12783467_0,hcc,hcc,washout,5.667513e-08,arterial phase enhancement,5.658390e-08,isointense on venous/delayed phase,5.656688e-08,enhancing rim/capsule,5.655837e-08,...,progressive enhancement,5.616085e-08,hyperenhancing mass on delayed phase,1.923924e-08,nodular growth,8.213609e-09,infiltrative growth,8.204522e-09,central scar,4.495069e-10
E106096969_0,hcc,hcc,washout,5.668720e-08,arterial phase enhancement,5.662793e-08,isointense on venous/delayed phase,5.656095e-08,enhancing rim/capsule,5.655961e-08,...,nodular or discontinuous enhancement,5.609333e-08,hyperenhancing mass on delayed phase,1.920656e-08,nodular growth,8.214523e-09,infiltrative growth,8.209521e-09,central scar,4.501344e-10
E105344747_0,hcc,colorectal,isointense on venous/delayed phase,5.640477e-08,arterial phase enhancement,5.637229e-08,enhancing rim/capsule,5.636788e-08,washout,5.632195e-08,...,hypoenhancing core,5.622293e-08,hyperenhancing mass on delayed phase,1.925245e-08,nodular growth,8.249595e-09,infiltrative growth,8.243143e-09,central scar,4.497236e-10
E101225606_0,hcc,colorectal,thin-walled,5.640039e-08,enhancing rim/capsule,5.639648e-08,isointense on venous/delayed phase,5.638757e-08,washout,5.638607e-08,...,heterogeneous lesion,5.621389e-08,hyperenhancing mass on delayed phase,1.926446e-08,nodular growth,8.238043e-09,infiltrative growth,8.225054e-09,central scar,4.487769e-10
E100407633_0,hcc,fnh,isointense on venous/delayed phase,5.675932e-08,arterial phase enhancement,5.669622e-08,washout,5.655479e-08,enhancing rim/capsule,5.651351e-08,...,heterogeneous lesion,5.612762e-08,hyperenhancing mass on delayed phase,1.924632e-08,nodular growth,8.206513e-09,infiltrative growth,8.206085e-09,central scar,4.516330e-10
E100262351_0,hcc,fnh,arterial phase enhancement,5.695555e-08,isointense on venous/delayed phase,5.682143e-08,washout,5.676809e-08,enhancing rim/capsule,5.663375e-08,...,progressive centripetal filling,5.612715e-08,hyperenhancing mass on delayed phase,1.922317e-08,infiltrative growth,8.229911e-09,nodular growth,8.220197e-09,central scar,4.546448e-10


### backup

In [15]:
model_num = 21
model_path = join(C.model_dir, "model_reader_new%d.hdf5" % model_num)

Z_reader = ['E103312835_1','12823036_0','12569915_0','E102093118_0','E102782525_0','12799652_0','E100894274_0','12874178_3','E100314676_0','12842070_0','13092836_2','12239783_0','12783467_0','13092966_0','E100962970_0','E100183257_1','E102634440_0','E106182827_0','12582632_0','E100121654_0','E100407633_0','E105310461_0','12788616_0','E101225606_0','12678910_1','E101083458_1','12324408_0','13031955_0','E101415263_0','E103192914_0','12888679_2','E106096969_0','E100192709_1','13112385_1','E100718398_0','12207268_0','E105244287_0','E102095465_0','E102613189_0','12961059_0','11907521_0','E105311123_0','12552705_0','E100610622_0','12975280_0','E105918926_0','E103020139_1','E101069048_1','E105427046_0','13028374_0','E100262351_0','12302576_0','12451831_0','E102929168_0','E100383453_0','E105344747_0','12569826_0','E100168661_0','12530153_0','E104697262_0']
orig_data_dict, num_samples = cbuild._collect_unaug_data()

features_by_cls, feat_count = cnna.collect_features()
feat_count.pop("homogeneous texture")
#feat_count.pop("central scar")
all_features = sorted(list(feat_count.keys()))
cls_features = {f: [c for c in C.cls_names if f in features_by_cls[c]] for f in all_features}

Z_features = cnna.get_annotated_files(features_by_cls)
Z_features.pop("homogeneous texture")
#Z_features.pop("central scar")

num_features = len(all_features)

voi_df = drm.get_voi_dfs()[0]
M = keras.models.load_model(model_path)

M.layers[5].get_weights()[1]

In [16]:
def memory():
    pid = os.getpid()
    py = psutil.Process(pid)
    print('Memory use:', py.memory_info()[0]/2.**30)

In [31]:
model_num = 26
model_path = join(C.model_dir, "model_reader_new%d.hdf5" % model_num)
inf_xls_path = 'D:\\feature_analysis\\influence%d.xlsx' % model_num

df = pd.DataFrame(columns=["true_cls", "pred_cls"] + all_features)
#df = pd.read_excel(inf_xls_path, index_col=0)

In [18]:
Z_full = np.concatenate([orig_data_dict[cls][-1] for cls in C.cls_names],0)

all_imgs = []
all_cls = []

for lesion_id in Z_full:
    cls = voi_df.loc[lesion_id]["cls"]
    img = np.load(join(C.orig_dir, cls, lesion_id+".npy"))
    all_imgs.append(np.expand_dims(img,0))
    all_cls.append(C.cls_names.index(cls))

In [32]:
importlib.reload(finf)
del M, IA
K.clear_session()
M = keras.models.load_model(model_path)
IA = finf.InfluenceAnalyzer(M, voi_df, all_imgs, all_cls)

In [ ]:
t = time.time()
for lesion_id in Z_reader:
    cls = voi_df.loc[lesion_id]["cls"]
    img = np.load(join(C.orig_dir, cls, lesion_id+".npy"))
    img = np.expand_dims(img,0)
    pred_cls = M.predict(img)[0]
    pred_cls = C.cls_names[list(pred_cls).index(pred_cls.max())]
        
    print(lesion_id)
    
    g_test = IA.get_grad(lesion_id, pred_cls=pred_cls)
    s_test = IA.get_stest(g_test)

    del M, IA
    K.clear_session()
    M = keras.models.load_model(model_path)
    IA = finf.InfluenceAnalyzer(M, voi_df, all_imgs, all_cls)

    I = {}
    for f in Z_features:
        I[f] = IA.get_avg_influence(Z_features[f], s_test)

    del M, IA
    K.clear_session()
    M = keras.models.load_model(model_path)
    IA = finf.InfluenceAnalyzer(M, voi_df, all_imgs, all_cls)
    
    df.loc[lesion_id] = [cls, pred_cls] + list(I[f] for f in df.columns[2:])

    print(time.time() - t)
    memory()

In [36]:
inf_xls_path

'D:\\feature_analysis\\influence26.xlsx'

In [37]:
df.to_excel(inf_xls_path)

In [ ]:
cls = voi_df.loc[lesion_id]["cls"]
x_test = np.load(join(C.orig_dir, cls, lesion_id+".npy"))
vis.draw_slices(x_test)

Memory leak test
for _ in range(10):
    memory()
    g_test = IA.get_grad(lesion_id)
memory()

### extra

In [2]:
def perturb_weights(W_true, t):
    eps = 1e-5
    W = copy.deepcopy(W_true)
    t_ix = 0
    
    for w_ix in range(len(W)):
        W[w_ix] += eps * np.reshape(t[t_ix:t_ix+W[w_ix].size], W[w_ix].shape)
        t_ix += W[w_ix].size
        
    return W

In [3]:
def get_grad(lesion_id, perturb_W=None):
    C = config.Config()
    
    cls = voi_df.loc[lesion_id]["cls"]
    img = np.load(join(C.orig_dir, cls, lesion_id+".npy"))
    img = np.expand_dims(img,0)

    y_true = np_utils.to_categorical(C.cls_names.index(cls), 6)
    loss = K.categorical_crossentropy(y_true, M.output)
    
    with tf.device('/gpu:0'):
        g = K.gradients(loss, M.trainable_weights)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        g_i = sess.run(g, feed_dict={M.input:img, K.learning_phase():0})
        if perturb_W is not None:
            feed_dict_plus = {M.trainable_weights[i]:perturb_W[i] for i in range(len(perturb_W))}
            g_i_plus = sess.run(g, feed_dict={**feed_dict_plus, M.input:img, K.learning_phase():0})

    g_i = np.concatenate([x.flatten() for x in g_i], 0)
    
    if perturb_W is not None:
        g_i_plus = np.concatenate([x.flatten() for x in g_i_plus], 0)
        return g_i, g_i_plus
    
    return g_i

In [4]:
def get_grads(Z_sample):
    eps = 1e-5
    Ht = np.zeros(g_test.shape)
    feed_dict_plus = {M.trainable_weights[i]:perturb_W[i] for i in range(len(perturb_W))}
    losses = [K.categorical_crossentropy(y_true, M.output) for y_true in \
              [np_utils.to_categorical(i, 6) for i in range(6)]]
        
    t = time.time()
    with tf.device('/gpu:0'):
        g = [K.gradients(loss, M.trainable_weights) for loss in losses]
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for ix in range(len(Z_sample)):
            g_i = sess.run(g[classes[ix]], feed_dict={M.input:imgs[ix], K.learning_phase():0})
            g_i = np.concatenate([x.flatten() for x in g_i], 0)
    if verbose:
        print(time.time()-t)
    
    return Ht / len(Z_sample)

In [24]:
def get_HVP(perturb_W, Z_sample, verbose=True):
    eps = 1e-5
    Ht = np.zeros(g_test.shape)
    feed_dict_plus = {M.trainable_weights[i]:perturb_W[i] for i in range(len(perturb_W))}
    losses = [K.categorical_crossentropy(y_true, M.output) for y_true in \
              [np_utils.to_categorical(i, 6) for i in range(6)]]
        
    t = time.time()
    with tf.device('/gpu:0'):
        g = [K.gradients(loss, M.trainable_weights) for loss in losses]
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for ix in range(len(Z_sample)):
            g_i = sess.run(g[classes[ix]], feed_dict={M.input:imgs[ix], K.learning_phase():0})
            g_i_plus = sess.run(g[classes[ix]], feed_dict={**feed_dict_plus, M.input:imgs[ix], K.learning_phase():0})
    
            g_i = np.concatenate([x.flatten() for x in g_i], 0)
            g_i_plus = np.concatenate([x.flatten() for x in g_i_plus], 0)

            Ht += (g_i_plus - g_i)/eps
    if verbose:
        print(time.time()-t)
    
    return Ht / len(Z_sample)

In [23]:
t_k = np.zeros(g_test.shape)
r_k = g_test#-Ht
p_k = r_k

In [ ]:
num_iters = 15
#phi_hist = []
t = time.time()
for _ in range(num_iters):
    #Z_sample = random.sample(list(Z_full), 15)
    W_new = perturb_weights(W, p_k)
    Hp = get_HVP(W_new, Z_full)

    alpha = np.dot(r_k, r_k) / np.dot(p_k, Hp)
    t_k += alpha * p_k
    r_k2 = r_k - alpha * Hp
    #phi_hist = .5*np.dot(t_k, Ht) - g_test

    beta = np.dot(r_k2, r_k2) / np.dot(r_k, r_k)
    r_k = r_k2
    p_k = r_k + beta*p_k
print(time.time()-t)

In [33]:
del p_k

In [36]:
def get_grads(imgs, classes, s_test, verbose=True):
    eps = 1e-5
    Ht = np.zeros(g_test.shape)
    losses = [K.categorical_crossentropy(y_true, M.output) for y_true in \
              [np_utils.to_categorical(i, 6) for i in range(6)]]
        
    I = 0
    t = time.time()
    with tf.device('/gpu:0'):
        g = [K.gradients(loss, M.trainable_weights) for loss in losses]
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for ix in range(len(classes)):
            g_i = sess.run(g[classes[ix]], feed_dict={M.input:imgs[ix], K.learning_phase():0})
            g_i = np.concatenate([x.flatten() for x in g_i], 0)
            I -= np.dot(g_i, s_test)
    if verbose:
        print(time.time()-t)
    
    return I / len(classes)

### others

In [47]:
df_0 = hf.pickle_load(join(C.base_dir, "data", "DFs4.bin"))[0]

In [41]:
data_dir = r"C:\Users\Clinton\Documents\voi-classifier\data"
answer_key = join(data_dir, "ground_truth.xlsx")
answer_key = pd.read_excel(answer_key, index_col=0)

In [46]:
answer_key.columns

Index(['true_cls', 'feature_1', 'feature_2', 'feature_3', 'feature_4',
       'ignore_1', 'ignore_2', 'BLANK', 'Unnamed: 9', 'Class', 'Num features'],
      dtype='object')

In [53]:
for z_test, row in answer_key.iterrows():
    cls = df_0.loc[z_test, "pred_cls"]
    feats = row[[f for f in answer_key.columns if f.startswith('feat')]].dropna().values
    break

In [58]:
feats.dropna().values

array(['nodular growth', 'heterogeneous lesion',
       'progressive enhancement'], dtype=object)